In [1]:
D.<x> = PolynomialRing(QQ)
R = PolynomialRing(QQ,'x')
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
Scplx= PowerSeriesRing(CC,'x',default_prec=300)
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m): 
    f=J(n,m)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m):
    return xjNewStep(n-1,m)
def xjNewCoefficient(n,m):
    if n>0:return (xjNew(n,m)-xjNew(n-1,m))/x^n
    if n==0:return 1
def xjNewTest(n,m):
    return D(xjNewStep(n-1,m))
def j(n,m):return xjNew(n,m)/x

def H4(n,m): # this is the definition in the published interpolations paper (2021):
    # def. 8.4
    jay=J(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-1)
    return ((S(numerator/denominator)^(1/(m-2))).O(n+1)).polynomial()
    #if argument is an S type then 'polynomial' takes no argument.

def H4strike(n,m): #this applies the strike operator to the version of H4 that *is*
    #in the Integers paper.
    poly=H4(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [0..polydegree])
    return answer


def H6(n,m):
    def base6(n,m):
        jay=J(n+1,m)
        num=(x*derivative(jay,x))^m # chain rule; variable is tau not x.
        den = expand(jay^(m-1)*(jay-1))
        return L((-1)^m*(num/den)).O(n+1)
    return S(base6(n,m)^(1/(m-2))).polynomial()
     #if argument is an S type then 'polynomial' takes no argument.

def H6strike(n,m): #this applies the strike operator to the version of H4 that *is*
    #in the Integers paper.
    poly=H6(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [0..polydegree])
    return answer

def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def nieburTau(n):
# The contortions below sidestep mysterious (to me) error messages...
# nieburTau is an algorith for Ramanujan''s tau from the paper
# "A formula for Ramanujan's tau-function" by Douglas Niebur.
# (He spells tau with the Greek letter.)
    def step1(n,k):
        return 35*k^4-52*k^3*n+18*k^2*n^2
    def step2(n,k):
        return sigma(k)*sigma(n-k)
    def step3(n):
        ans=0
        for k in [1..n-1]:
            ans=ans+step1(n,k)*step2(n,k)
        return 24*ans
    return n^4*sigma(n)-step3(n)

def deltaDagger(n,m):
    def ddstep1(n,m):
        return D(S(expand((H4(n+2,m)^3)))).truncate(n+1)
    def ddstep2(n,m):
        return D(S((H6(n+2,m)^2))).truncate(n+1)
    return ((ddstep1(n,m)-ddstep2(n,m))/1728)

def deltaDaggerStrike(n,m): #this applies the strike operator to the version of H4 that *is*
    #in the Integers paper.
    poly=deltaDagger(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [1..polydegree])
    return answer

def deltaDagger2(n,m):
    def ddstep1(n,m):
        return D(S(expand((H4(n+2,m)^3)))).truncate(n+1)
    def ddstep2(n,m):
        return D(S((H6(n+2,m)^2))).truncate(n+1)
    return (ddstep1(n,m)-ddstep2(n,m))

# def deltaDaggerStrike(n,m):
 #   return S(((H4strike(n,3)^3-H6strike(n,3)^2)/1728)).truncate(n+1)
    
def deltaDagger2Strike(n,m): #this applies the strike operator to the version of H4 that *is*
    #in the Integers paper.
    poly=deltaDagger2(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [1..polydegree])
    return answer

def first(list):return list[0]
def last(list):return list[-1]

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr



print("ok1")

ok1


In [2]:
data=[]
import time
start=time.time()/60
for m in [3..1203]:
    poly=H4(200,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/24sept22no1.txt','wb') # H4 series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[3, 200, 0.27416178584098816]
[4, 200, 0.4724309481680393]
[5, 200, 0.8450320325791836]
[6, 199, 1.0186402350664139]
[7, 200, 1.4207725822925568]
[8, 200, 1.7371144481003284]
[9, 200, 2.1730619184672832]
[10, 200, 2.452136803418398]
[11, 200, 2.9029116332530975]
[12, 200, 3.285788781940937]
[13, 200, 3.7538143694400787]
[14, 200, 4.057925716042519]
[15, 200, 4.588935501873493]
[16, 200, 4.955913379788399]
[17, 200, 5.455510836094618]
[18, 200, 5.793895915150642]
[19, 200, 6.281520381569862]
[20, 200, 6.718764949589968]
[21, 200, 7.264707986265421]
[22, 200, 7.592765718698502]
[23, 200, 8.091646380722523]
[24, 200, 8.550945565104485]
[25, 200, 9.102406837046146]
[26, 200, 9.475031901150942]
[27, 200, 10.025452714413404]
[28, 200, 10.49638594686985]
[29, 200, 11.017314501106739]
[30, 200, 11.448775451630354]
[31, 200, 11.988834667950869]
[32, 200, 12.422229766845703]
[33, 200, 13.02582198008895]
[34, 200, 13.415049102157354]
[35, 200, 14.029502864927053]
[36, 200, 14.534267418086529]
[37

[276, 200, 163.1564825810492]
[277, 200, 163.840851534158]
[278, 200, 164.36948639899492]
[279, 200, 165.1023448817432]
[280, 200, 165.787160217762]
[281, 200, 166.46871137991548]
[282, 200, 167.04714464768767]
[283, 200, 167.74500216543674]
[284, 200, 168.37612803280354]
[285, 200, 169.15717351809144]
[286, 200, 169.70692430064082]
[287, 200, 170.4515691176057]
[288, 200, 171.08008040115237]
[289, 200, 171.8162242025137]
[290, 200, 172.38228625059128]
[291, 200, 173.11830543354154]
[292, 200, 173.73256599903107]
[293, 200, 174.43450728803873]
[294, 200, 175.03038453310728]
[295, 200, 175.76504101231694]
[296, 200, 176.3877769522369]
[297, 200, 177.12991078197956]
[298, 200, 177.6528933495283]
[299, 200, 178.38555723428726]
[300, 200, 179.07191678509116]
[301, 200, 179.8109517507255]
[302, 200, 180.32818394899368]
[303, 200, 181.0726052634418]
[304, 200, 181.6973763294518]
[305, 200, 182.44323340058327]
[306, 200, 183.02239498496056]
[307, 200, 183.73715993389487]
[308, 200, 184.417964

[545, 200, 350.93124245107174]
[546, 200, 351.5899532176554]
[547, 200, 352.32521548494697]
[548, 200, 353.02225971594453]
[549, 200, 353.8400243334472]
[550, 200, 354.49583265185356]
[551, 200, 355.2836530543864]
[552, 200, 356.01176380366087]
[553, 200, 356.7948293350637]
[554, 200, 357.38570276647806]
[555, 200, 358.23886663094163]
[556, 200, 358.9094224348664]
[557, 200, 359.6900451146066]
[558, 200, 360.3220274336636]
[559, 200, 361.1198925189674]
[560, 200, 361.8510499857366]
[561, 200, 362.6914247535169]
[562, 200, 363.27954242005944]
[563, 200, 364.0408998169005]
[564, 200, 364.78180005028844]
[565, 200, 365.57714883610606]
[566, 200, 366.1889613009989]
[567, 200, 367.02713026478887]
[568, 200, 367.71582105383277]
[569, 200, 368.4526848681271]
[570, 200, 369.1238401122391]
[571, 200, 369.8903124816716]
[572, 200, 370.60803316533566]
[573, 200, 371.4137075319886]
[574, 200, 372.03680542111397]
[575, 200, 372.86108550056815]
[576, 200, 373.56725941970944]
[577, 200, 374.337092749

[816, 200, 553.588692266494]
[817, 200, 554.4176734015346]
[818, 200, 555.0607360526919]
[819, 200, 555.9383602514863]
[820, 200, 556.7238066829741]
[821, 200, 557.5182987526059]
[822, 200, 558.1825708672404]
[823, 200, 558.982055015862]
[824, 200, 559.7135923504829]
[825, 200, 560.6077813357115]
[826, 200, 561.2617557160556]
[827, 200, 562.0590283349156]
[828, 200, 562.8177924007177]
[829, 200, 563.6162765361369]
[830, 200, 564.2713638320565]
[831, 200, 565.0901037491858]
[832, 200, 565.8043105639517]
[833, 200, 566.6452562324703]
[834, 200, 567.3219747133553]
[835, 200, 568.1385013870895]
[836, 200, 568.910048365593]
[837, 200, 569.7517300024629]
[838, 200, 570.3910707831383]
[839, 200, 571.1771837323904]
[840, 200, 572.0070804022253]
[841, 200, 572.8026998825371]
[842, 200, 573.4148678816855]
[843, 200, 574.2559204027057]
[844, 200, 574.9678421504796]
[845, 200, 575.8365348987281]
[846, 200, 576.5096940845251]
[847, 200, 577.3611364997923]
[848, 200, 578.061280850321]
[849, 200, 578

[1088, 200, 767.3963010683656]
[1089, 200, 768.2909963652492]
[1090, 200, 768.9947427511215]
[1091, 200, 769.8230711333454]
[1092, 200, 770.6542230509222]
[1093, 200, 771.4776354692876]
[1094, 200, 772.1215303502977]
[1095, 200, 773.0225973315537]
[1096, 200, 773.7658547312021]
[1097, 200, 774.6012787148356]
[1098, 200, 775.2971204482019]
[1099, 200, 776.1697961166501]
[1100, 200, 776.9951781332493]
[1101, 200, 777.8660036362708]
[1102, 200, 778.5254362821579]
[1103, 200, 779.3497456870973]
[1104, 200, 780.1415609009564]
[1105, 200, 781.0427191853523]
[1106, 200, 781.7409941144288]
[1107, 200, 782.6389071047306]
[1108, 200, 783.4069381020963]
[1109, 200, 784.2529462836683]
[1110, 200, 784.995759498328]
[1111, 200, 785.8604651987553]
[1112, 200, 786.6140493638813]
[1113, 200, 787.5278683193028]
[1114, 200, 788.1933205835521]
[1115, 200, 789.0932475812733]
[1116, 200, 789.9088051468134]
[1117, 200, 790.7646413818002]
[1118, 200, 791.4439085163176]
[1119, 200, 792.3036087527871]
[1120, 20

In [3]:
data=[]
import time
start=time.time()/60
for m in [1204..2403]:
    poly=H4(200,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/25sept22no2.txt','wb') # H4 series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[1204, 200, 0.7923523187637329]
[1205, 200, 1.6813494861125946]
[1206, 200, 2.366381950676441]
[1207, 200, 3.2393316850066185]
[1208, 200, 3.99722708389163]
[1209, 200, 4.917757283896208]
[1210, 200, 5.632594618946314]
[1211, 200, 6.503068417310715]
[1212, 200, 7.313473518937826]
[1213, 200, 8.150695066899061]
[1214, 200, 8.822927683591843]
[1215, 200, 9.7435133010149]
[1216, 200, 10.509187284857035]
[1217, 200, 11.332779705524445]
[1218, 200, 12.080927465111017]
[1219, 200, 12.918236717581749]
[1220, 200, 13.715212769806385]
[1221, 200, 14.611278966069221]
[1222, 200, 15.277897216379642]
[1223, 200, 16.12558653205633]
[1224, 200, 16.952985119074583]
[1225, 200, 17.876931749284267]
[1226, 200, 18.52809651941061]
[1227, 200, 19.38734283298254]
[1228, 200, 20.1421060860157]
[1229, 200, 20.972431231290102]
[1230, 200, 21.720818366855383]
[1231, 200, 22.541507732123137]
[1232, 200, 23.36180728301406]
[1233, 200, 24.25117328390479]
[1234, 200, 24.943470250815153]
[1235, 200, 25.857061285525

[1465, 200, 215.70292698219419]
[1466, 200, 216.40096208453178]
[1467, 200, 217.29445076733828]
[1468, 200, 218.09783086553216]
[1469, 200, 218.97279405221343]
[1470, 200, 219.76253103092313]
[1471, 200, 220.6290975511074]
[1472, 200, 221.37735906615853]
[1473, 200, 222.27292161807418]
[1474, 200, 222.97398616746068]
[1475, 200, 223.89043763279915]
[1476, 200, 224.71423096582294]
[1477, 200, 225.6030932329595]
[1478, 200, 226.27095115184784]
[1479, 200, 227.18952311947942]
[1480, 200, 228.00767148286104]
[1481, 200, 228.86019380018115]
[1482, 200, 229.61549043655396]
[1483, 200, 230.483364533633]
[1484, 200, 231.31130518391728]
[1485, 200, 232.2829822152853]
[1486, 200, 232.97826255485415]
[1487, 200, 233.84843154996634]
[1488, 200, 234.66832296550274]
[1489, 200, 235.51134526729584]
[1490, 200, 236.21469279751182]
[1491, 200, 237.15715769678354]
[1492, 200, 237.9471725858748]
[1493, 200, 238.81713965162635]
[1494, 200, 239.53583331778646]
[1495, 200, 240.47988018020988]
[1496, 200, 24

[1726, 200, 433.0647944509983]
[1727, 200, 433.96403670310974]
[1728, 200, 434.7581253051758]
[1729, 200, 435.70314928516746]
[1730, 200, 436.4286535009742]
[1731, 200, 437.32976780086756]
[1732, 200, 438.13143768161535]
[1733, 200, 439.00486209988594]
[1734, 200, 439.7545736543834]
[1735, 200, 440.66882706806064]
[1736, 200, 441.50970516726375]
[1737, 200, 442.4310930520296]
[1738, 200, 443.16290716826916]
[1739, 200, 444.05885500088334]
[1740, 200, 444.9400027655065]
[1741, 200, 445.83688398450613]
[1742, 200, 446.55232805013657]
[1743, 200, 447.50539055094123]
[1744, 200, 448.27683666720986]
[1745, 200, 449.199131667614]
[1746, 200, 449.93943725153804]
[1747, 200, 450.8331122137606]
[1748, 200, 451.65563398599625]
[1749, 200, 452.6071868017316]
[1750, 200, 453.3490899465978]
[1751, 200, 454.2518211454153]
[1752, 200, 455.07055696472526]
[1753, 200, 455.94090908765793]
[1754, 200, 456.6450389325619]
[1755, 200, 457.6185704804957]
[1756, 200, 458.4057428650558]
[1757, 200, 459.3075211

[1991, 200, 658.0519022643566]
[1992, 200, 658.9086870513856]
[1993, 200, 659.7983123660088]
[1994, 200, 660.4901307485998]
[1995, 200, 661.5064396858215]
[1996, 200, 662.3081084676087]
[1997, 200, 663.1977000981569]
[1998, 200, 663.9505208842456]
[1999, 200, 664.849956586957]
[2000, 200, 665.6808352135122]
[2001, 200, 666.6331135854125]
[2002, 200, 667.3954492211342]
[2003, 200, 668.2644979171455]
[2004, 200, 669.1105368994176]
[2005, 200, 670.025124501437]
[2006, 200, 670.7455762177706]
[2007, 200, 671.6714156158268]
[2008, 200, 672.4938563816249]
[2009, 200, 673.4249898344278]
[2010, 200, 674.2204786688089]
[2011, 200, 675.0992927327752]
[2012, 200, 675.9127749986947]
[2013, 200, 676.8715540505946]
[2014, 200, 677.5841948017478]
[2015, 200, 678.5322110466659]
[2016, 200, 679.387389164418]
[2017, 200, 680.288052663207]
[2018, 200, 681.0042514838278]
[2019, 200, 681.9282203875482]
[2020, 200, 682.7598320841789]
[2021, 200, 683.6391845345497]
[2022, 200, 684.375330299139]
[2023, 200, 6

[2257, 200, 887.2570181488991]
[2258, 200, 887.9859167672694]
[2259, 200, 888.948886834085]
[2260, 200, 889.8216537497938]
[2261, 200, 890.794573970139]
[2262, 200, 891.5779658667743]
[2263, 200, 892.5037646330893]
[2264, 200, 893.326567016542]
[2265, 200, 894.3273313976824]
[2266, 200, 895.0783762820065]
[2267, 200, 896.0030945017934]
[2268, 200, 896.9146546684206]
[2269, 200, 897.8306402154267]
[2270, 200, 898.5915834009647]
[2271, 200, 899.5425716675818]
[2272, 200, 900.3340977542102]
[2273, 200, 901.2148717343807]
[2274, 200, 901.9923918135464]
[2275, 200, 902.9842288829386]
[2276, 200, 903.8328475356102]
[2277, 200, 904.805519066751]
[2278, 200, 905.5415647961199]
[2279, 200, 906.445155415684]
[2280, 200, 907.3579173982143]
[2281, 200, 908.2605083175004]
[2282, 200, 909.0209316834807]
[2283, 200, 909.9710523672402]
[2284, 200, 910.7905787676573]
[2285, 200, 911.7326757013798]
[2286, 200, 912.5067208372056]
[2287, 200, 913.4222918488085]
[2288, 200, 914.2746597826481]
[2289, 200, 9

In [2]:
data=[]
import time
start=time.time()/60
for m in [2404..3610]:
    poly=H4(100,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/12sept22no9.txt','wb') # H4 series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[2404, 100, 0.057583048939704895]
[2405, 100, 0.12206731736660004]
[2406, 100, 0.17468643561005592]
[2407, 100, 0.23686020076274872]
[2408, 100, 0.2939843684434891]
[2409, 100, 0.35994651913642883]
[2410, 100, 0.412408284842968]
[2411, 100, 0.4761541858315468]
[2412, 100, 0.536301001906395]
[2413, 100, 0.600569985806942]
[2414, 100, 0.6511084884405136]
[2415, 100, 0.7211095169186592]
[2416, 100, 0.7779682353138924]
[2417, 100, 0.8418632857501507]
[2418, 100, 0.8977376706898212]
[2419, 100, 0.9612654186785221]
[2420, 100, 1.0227625519037247]
[2421, 100, 1.0888375006616116]
[2422, 100, 1.142305750399828]
[2423, 100, 1.2054380178451538]
[2424, 100, 1.2651423029601574]
[2425, 100, 1.330748837441206]
[2426, 100, 1.3819874003529549]
[2427, 100, 1.4481214880943298]
[2428, 100, 1.5054202154278755]
[2429, 100, 1.571230635046959]
[2430, 100, 1.6279001533985138]
[2431, 100, 1.6941123344004154]
[2432, 100, 1.7477945014834404]
[2433, 100, 1.8150046691298485]
[2434, 100, 1.8666271679103374]
[2435, 1

[2665, 100, 15.83124453574419]
[2666, 100, 15.881765604019165]
[2667, 100, 15.947864335030317]
[2668, 100, 16.00460458174348]
[2669, 100, 16.065972838550806]
[2670, 100, 16.121454935520887]
[2671, 100, 16.18180636689067]
[2672, 100, 16.2363738194108]
[2673, 100, 16.301008701324463]
[2674, 100, 16.353462252765894]
[2675, 100, 16.41707943379879]
[2676, 100, 16.475990667939186]
[2677, 100, 16.536921352148056]
[2678, 100, 16.588120851665735]
[2679, 100, 16.653887066990137]
[2680, 100, 16.71198983117938]
[2681, 100, 16.775743134319782]
[2682, 100, 16.829134203493595]
[2683, 100, 16.892338570207357]
[2684, 100, 16.95007411763072]
[2685, 100, 17.0170963332057]
[2686, 100, 17.068523600697517]
[2687, 100, 17.130451533943415]
[2688, 100, 17.18723126500845]
[2689, 100, 17.248375300318003]
[2690, 100, 17.301176887005568]
[2691, 100, 17.3670520670712]
[2692, 100, 17.423254001885653]
[2693, 100, 17.484671119600534]
[2694, 100, 17.536705002188683]
[2695, 100, 17.603458385914564]
[2696, 100, 17.658554

[2924, 100, 31.177067786455154]
[2925, 100, 31.245678052306175]
[2926, 100, 31.300815533846617]
[2927, 100, 31.361923284828663]
[2928, 100, 31.41985570266843]
[2929, 100, 31.48203395307064]
[2930, 100, 31.535295318812132]
[2931, 100, 31.600071419030428]
[2932, 100, 31.656494837254286]
[2933, 100, 31.7217849008739]
[2934, 100, 31.774711303412914]
[2935, 100, 31.84025977179408]
[2936, 100, 31.89605788514018]
[2937, 100, 31.96223260089755]
[2938, 100, 32.01350258663297]
[2939, 100, 32.07613863423467]
[2940, 100, 32.13985065370798]
[2941, 100, 32.202399134635925]
[2942, 100, 32.25374545156956]
[2943, 100, 32.31813285499811]
[2944, 100, 32.370876383036375]
[2945, 100, 32.43657151982188]
[2946, 100, 32.48865208774805]
[2947, 100, 32.55163539946079]
[2948, 100, 32.60852498561144]
[2949, 100, 32.6741137355566]
[2950, 100, 32.7267380990088]
[2951, 100, 32.79085498675704]
[2952, 100, 32.84958961978555]
[2953, 100, 32.912225637584925]
[2954, 100, 32.963601000607014]
[2955, 100, 33.03119185566902]

[3187, 100, 46.97956934943795]
[3188, 100, 47.035990349948406]
[3189, 100, 47.1016945540905]
[3190, 100, 47.156762667000294]
[3191, 100, 47.21896160021424]
[3192, 100, 47.28069945052266]
[3193, 100, 47.34354588389397]
[3194, 100, 47.39493430405855]
[3195, 100, 47.46303520351648]
[3196, 100, 47.52153253555298]
[3197, 100, 47.583582635968924]
[3198, 100, 47.63830640166998]
[3199, 100, 47.70316741988063]
[3200, 100, 47.75912259891629]
[3201, 100, 47.826564718037844]
[3202, 100, 47.87854568660259]
[3203, 100, 47.94220921769738]
[3204, 100, 48.00010196864605]
[3205, 100, 48.06600485369563]
[3206, 100, 48.11894318833947]
[3207, 100, 48.18463172018528]
[3208, 100, 48.240460488945246]
[3209, 100, 48.30268841981888]
[3210, 100, 48.35929337143898]
[3211, 100, 48.42481108382344]
[3212, 100, 48.48481595143676]
[3213, 100, 48.55285553634167]
[3214, 100, 48.604883566498756]
[3215, 100, 48.66910423338413]
[3216, 100, 48.72835476696491]
[3217, 100, 48.79130640625954]
[3218, 100, 48.84167406708002]
[32

[3450, 100, 62.86961745098233]
[3451, 100, 62.93518368527293]
[3452, 100, 62.99284600093961]
[3453, 100, 63.059073466807604]
[3454, 100, 63.111812233924866]
[3455, 100, 63.17884248495102]
[3456, 100, 63.23502951860428]
[3457, 100, 63.300126768648624]
[3458, 100, 63.35558916628361]
[3459, 100, 63.42216903343797]
[3460, 100, 63.481430884450674]
[3461, 100, 63.544829186052084]
[3462, 100, 63.5991881005466]
[3463, 100, 63.66283733397722]
[3464, 100, 63.721018966287374]
[3465, 100, 63.792394787073135]
[3466, 100, 63.845029801130295]
[3467, 100, 63.90882573649287]
[3468, 100, 63.97095053270459]
[3469, 100, 64.03424728289247]
[3470, 100, 64.08845899999142]
[3471, 100, 64.15449609607458]
[3472, 100, 64.21109085157514]
[3473, 100, 64.27500034868717]
[3474, 100, 64.3296850733459]
[3475, 100, 64.39574256911874]
[3476, 100, 64.45460933819413]
[3477, 100, 64.52018146589398]
[3478, 100, 64.57184168696404]
[3479, 100, 64.63597295060754]
[3480, 100, 64.69954008236527]
[3481, 100, 64.76182716712356]
[3

In [4]:
data=[]
import time
start=time.time()/60
for m in [3611..4818]:
    poly=H4(100,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/12sept22no10.txt','wb') # H4 series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[3611, 100, 0.06645430624485016]
[3612, 100, 0.1310855709016323]
[3613, 100, 0.19622565433382988]
[3614, 100, 0.2505277320742607]
[3615, 100, 0.32101158797740936]
[3616, 100, 0.3783796355128288]
[3617, 100, 0.44484832137823105]
[3618, 100, 0.5001235865056515]
[3619, 100, 0.5680827833712101]
[3620, 100, 0.6284056529402733]
[3621, 100, 0.6969343349337578]
[3622, 100, 0.7491402365267277]
[3623, 100, 0.8132683038711548]
[3624, 100, 0.8736147321760654]
[3625, 100, 0.9412079490721226]
[3626, 100, 0.9973025321960449]
[3627, 100, 1.0657503195106983]
[3628, 100, 1.123953353613615]
[3629, 100, 1.1887482553720474]
[3630, 100, 1.2472237050533295]
[3631, 100, 1.3120440356433392]
[3632, 100, 1.3695291206240654]
[3633, 100, 1.4392768554389477]
[3634, 100, 1.4916533716022968]
[3635, 100, 1.5594743862748146]
[3636, 100, 1.623686883598566]
[3637, 100, 1.6900700852274895]
[3638, 100, 1.744690652936697]
[3639, 100, 1.8135329186916351]
[3640, 100, 1.876866102218628]
[3641, 100, 1.94306705519557]
[3642, 100

[3871, 100, 16.337689865380526]
[3872, 100, 16.394837886095047]
[3873, 100, 16.46095246449113]
[3874, 100, 16.514296885579824]
[3875, 100, 16.58037518709898]
[3876, 100, 16.643674451857805]
[3877, 100, 16.706451438367367]
[3878, 100, 16.76146014034748]
[3879, 100, 16.82803237065673]
[3880, 100, 16.88958388566971]
[3881, 100, 16.95376978442073]
[3882, 100, 17.00920147076249]
[3883, 100, 17.074567820876837]
[3884, 100, 17.13296226784587]
[3885, 100, 17.204718735069036]
[3886, 100, 17.258446589112282]
[3887, 100, 17.326202020049095]
[3888, 100, 17.387248903512955]
[3889, 100, 17.45316280424595]
[3890, 100, 17.50684031844139]
[3891, 100, 17.57473075389862]
[3892, 100, 17.635789405554533]
[3893, 100, 17.700829904526472]
[3894, 100, 17.757517755031586]
[3895, 100, 17.826252173632383]
[3896, 100, 17.88542353734374]
[3897, 100, 17.952634371817112]
[3898, 100, 18.006328098475933]
[3899, 100, 18.07261797040701]
[3900, 100, 18.13857515528798]
[3901, 100, 18.20300366729498]
[3902, 100, 18.25628432

[4130, 100, 32.491991970688105]
[4131, 100, 32.55992357060313]
[4132, 100, 32.620982218533754]
[4133, 100, 32.6859747171402]
[4134, 100, 32.74375203624368]
[4135, 100, 32.810073517262936]
[4136, 100, 32.870372001081705]
[4137, 100, 32.9398183375597]
[4138, 100, 32.99341280385852]
[4139, 100, 33.059806019067764]
[4140, 100, 33.12546616792679]
[4141, 100, 33.192030902951956]
[4142, 100, 33.245999187231064]
[4143, 100, 33.31437600031495]
[4144, 100, 33.3736192882061]
[4145, 100, 33.44065450131893]
[4146, 100, 33.49560375511646]
[4147, 100, 33.5624032728374]
[4148, 100, 33.623218171298504]
[4149, 100, 33.69141515344381]
[4150, 100, 33.74802194908261]
[4151, 100, 33.81553381681442]
[4152, 100, 33.87674067169428]
[4153, 100, 33.94109412282705]
[4154, 100, 33.99456823989749]
[4155, 100, 34.06482323631644]
[4156, 100, 34.12346035242081]
[4157, 100, 34.189471285790205]
[4158, 100, 34.24869788810611]
[4159, 100, 34.31411687284708]
[4160, 100, 34.37290807068348]
[4161, 100, 34.44123726710677]
[41

[4393, 100, 49.33199882134795]
[4394, 100, 49.3881498016417]
[4395, 100, 49.459729351103306]
[4396, 100, 49.52069940418005]
[4397, 100, 49.587106954306364]
[4398, 100, 49.64441345259547]
[4399, 100, 49.71283773705363]
[4400, 100, 49.77688816934824]
[4401, 100, 49.84740265458822]
[4402, 100, 49.90185236558318]
[4403, 100, 49.969567373394966]
[4404, 100, 50.031575452536345]
[4405, 100, 50.099559653550386]
[4406, 100, 50.1512825191021]
[4407, 100, 50.22062838450074]
[4408, 100, 50.28245495259762]
[4409, 100, 50.350133039057255]
[4410, 100, 50.411538023501635]
[4411, 100, 50.47976065054536]
[4412, 100, 50.539283752441406]
[4413, 100, 50.6073826700449]
[4414, 100, 50.66213405132294]
[4415, 100, 50.73056370019913]
[4416, 100, 50.79117250442505]
[4417, 100, 50.86010563373566]
[4418, 100, 50.91506129875779]
[4419, 100, 50.98340658470988]
[4420, 100, 51.04877212271094]
[4421, 100, 51.11532463878393]
[4422, 100, 51.17366560548544]
[4423, 100, 51.239211335778236]
[4424, 100, 51.30192243307829]
[4

[4657, 100, 66.50100810453296]
[4658, 100, 66.56016036868095]
[4659, 100, 66.63527963683009]
[4660, 100, 66.70064063742757]
[4661, 100, 66.77076283469796]
[4662, 100, 66.83208338543773]
[4663, 100, 66.90177068859339]
[4664, 100, 66.96599028632045]
[4665, 100, 67.04083511605859]
[4666, 100, 67.0978737026453]
[4667, 100, 67.16824625432491]
[4668, 100, 67.23560748621821]
[4669, 100, 67.30669478699565]
[4670, 100, 67.36655196920037]
[4671, 100, 67.43786823377013]
[4672, 100, 67.49861742183566]
[4673, 100, 67.56643759831786]
[4674, 100, 67.62692768871784]
[4675, 100, 67.70009023323655]
[4676, 100, 67.76475564017892]
[4677, 100, 67.83571480214596]
[4678, 100, 67.89130946993828]
[4679, 100, 67.96028103679419]
[4680, 100, 68.02810930088162]
[4681, 100, 68.09699783474207]
[4682, 100, 68.15277160704136]
[4683, 100, 68.2252810522914]
[4684, 100, 68.28742491826415]
[4685, 100, 68.35710295289755]
[4686, 100, 68.41793708503246]
[4687, 100, 68.48652182146907]
[4688, 100, 68.54826213419437]
[4689, 100